# FIA to FVS Forest Growth Simulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mihiarc/pyfvs/blob/main/notebooks/fia_to_fvs_colab.ipynb)

This notebook demonstrates the complete workflow for simulating forest growth using real FIA (Forest Inventory and Analysis) data with PyFVS (Python Forest Vegetation Simulator).

## FIA Python Ecosystem

This notebook uses two packages from the FIA Python Ecosystem:

- **[PyFIA](https://github.com/mihiarc/pyfia)**: Survey/plot data analysis - loads and queries FIA databases
- **[PyFVS](https://github.com/mihiarc/pyfvs)**: Growth/yield simulation - simulates forest growth over time

## Workflow Overview

1. **Install packages** and download FIA data
2. **Load FIA data** using PyFIA
3. **Create a PyFVS Stand** from FIA plot data
4. **Simulate growth** for 25+ years
5. **Generate yield tables** and visualizations

## Step 1: Install Packages

First, we'll install PyFIA and FVS-Python from PyPI.

In [ ]:
# Install the FIA Python Ecosystem packages from PyPI
!pip install -q fvs-python pyfia rich

# Verify installation
import pyfvs
print(f"fvs-python {pyfvs.__version__} installed successfully!")
print("Imports as: from pyfvs import Stand")

## Step 2: Download FIA Database

PyFIA can download FIA databases for any US state directly from the FIA DataMart. We'll download North Carolina (NC) data for this example.

**Note:** The download may take a few minutes depending on the state size and internet connection.

In [ ]:
# Download FIA database for North Carolina
from pyfia import download

# Download returns the path to the DuckDB database
# Data is saved to ~/.pyfia/data/nc/nc.duckdb
DB_PATH = download("NC")

print(f"\nFIA database downloaded to: {DB_PATH}")

## Step 3: Import Libraries and Set Up

In [ ]:
import polars as pl
import matplotlib.pyplot as plt

# PyFIA for loading FIA data
from pyfia import FIA

# PyFVS for growth simulation
from pyfvs import Stand
from pyfvs.fia_integration import FIASpeciesMapper

# DB_PATH was set in Step 2 by the download() function
STATE_FIPS = 37  # North Carolina

print(f"Database path: {DB_PATH}")
print(f"Database exists: {DB_PATH.exists()}")

## Step 4: Load FIA Data with PyFIA

We'll connect to the FIA database and extract tree and condition data for analysis.

In [ ]:
# Connect to FIA database and load data
with FIA(str(DB_PATH)) as db:
    # Clip to state and most recent inventory
    db.clip_by_state(STATE_FIPS)
    db.clip_most_recent(eval_type="VOL")
    
    # Load PLOT table first (required for EVALID filtering)
    db.load_table("PLOT")
    
    # Get all trees with required columns for FVS
    trees = db.get_trees(columns=[
        "CN", "PLT_CN", "SUBP", "TREE", "CONDID",
        "SPCD", "DIA", "HT", "CR",
        "TPA_UNADJ", "STATUSCD", "TOTAGE"
    ])
    
    # Get all conditions
    conds = db.get_conditions(columns=[
        "CN", "PLT_CN", "CONDID",
        "SICOND", "SIBASE", "SISP",
        "FORTYPCD", "STDAGE", "STDSZCD",
        "SITECLCD", "OWNGRPCD",
        "CONDPROP_UNADJ", "COND_STATUS_CD"
    ])

print(f"Loaded {len(trees):,} trees from {trees['PLT_CN'].n_unique():,} plots")
print(f"Loaded {len(conds):,} conditions")

## Step 5: Find a Good Loblolly Pine Plot

Let's find a plot dominated by loblolly pine (SPCD=131) for our simulation.

In [ ]:
# FIA species codes for southern pines
PINE_SPECIES = {
    131: "Loblolly pine",
    110: "Shortleaf pine",
    121: "Longleaf pine",
    111: "Slash pine"
}

TARGET_SPECIES = 131  # Loblolly pine
MIN_TREES = 10  # Minimum trees of target species

# Find plots with good loblolly pine representation
good_plots = (
    trees
    .filter(pl.col("SPCD") == TARGET_SPECIES)
    .filter(pl.col("STATUSCD") == 1)  # Live trees only
    .filter(pl.col("DIA") >= 5.0)  # Merchantable size
    .group_by("PLT_CN")
    .agg([
        pl.len().alias("n_target"),
        pl.col("DIA").mean().alias("mean_dia"),
        pl.col("TPA_UNADJ").sum().alias("total_tpa")
    ])
    .filter(pl.col("n_target") >= MIN_TREES)
    .sort("n_target", descending=True)
)

print(f"Found {len(good_plots):,} plots with {MIN_TREES}+ {PINE_SPECIES[TARGET_SPECIES]} trees")
print("\nTop 5 plots by tree count:")
print(good_plots.head(5))

In [ ]:
# Select the best plot
selected_plot_cn = good_plots["PLT_CN"][0]
print(f"Selected plot CN: {selected_plot_cn}")

# Filter to selected plot
plot_trees = trees.filter(pl.col("PLT_CN") == selected_plot_cn)
plot_conds = conds.filter(pl.col("PLT_CN") == selected_plot_cn)

print(f"\nPlot has {len(plot_trees)} trees in {len(plot_conds)} condition(s)")

# Show species composition
mapper = FIASpeciesMapper()
species_summary = (
    plot_trees
    .filter(pl.col("STATUSCD") == 1)
    .group_by("SPCD")
    .agg([pl.len().alias("count"), pl.col("DIA").mean().alias("avg_dia")])
    .sort("count", descending=True)
)

print("\nSpecies composition:")
for row in species_summary.iter_rows(named=True):
    spcd = int(row["SPCD"])
    name = mapper.get_common_name(spcd) or f"Unknown ({spcd})"
    print(f"  {name}: {row['count']} trees, avg DBH: {row['avg_dia']:.1f} in")

## Step 6: Create PyFVS Stand from FIA Data

Now we'll convert the FIA plot data into a PyFVS Stand object. The `Stand.from_fia_data()` method handles:

- **Species code conversion**: FIA SPCD (e.g., 131) → FVS 2-letter code (e.g., "LP")
- **Crown ratio conversion**: Percentage (0-100) → Proportion (0-1)
- **Site index derivation**: From SICOND column or default
- **Forest type mapping**: From FORTYPCD or species composition
- **TPA weighting**: Expands trees based on TPA_UNADJ values

In [ ]:
# Create PyFVS Stand from FIA data
stand = Stand.from_fia_data(
    tree_df=plot_trees,
    cond_df=plot_conds,
    # Optional parameters:
    # site_index=70,        # Override site index if known
    # condid=1,             # Select specific condition
    # ecounit="M231",       # Set ecological unit for growth modifiers
    # forest_type="FTYLPN", # Override forest type
    weight_by_tpa=True,     # Replicate trees based on TPA_UNADJ
    min_dia=1.0,            # Minimum diameter threshold
    max_trees=1000,         # Maximum trees (subsample if exceeded)
    condition_strategy="dominant"  # Strategy for multi-condition plots
)

print(f"Stand created successfully!")
print(f"  Trees in stand: {len(stand.trees)}")
print(f"  Site Index: {stand.site_index} feet (base age 25)")
print(f"  Forest Type: {stand.forest_type}")
print(f"  Dominant Species: {stand.species}")
print(f"  Stand Age: {stand.age} years")

## Step 7: Display Initial Stand Metrics

Before simulating growth, let's examine the current stand conditions.

In [ ]:
def display_metrics(stand, title="Stand Metrics"):
    """Display stand metrics in a formatted way."""
    metrics = stand.get_metrics()
    
    print(f"\n{'='*50}")
    print(f"{title:^50}")
    print(f"{'='*50}")
    print(f"{'Structural Metrics':^50}")
    print(f"-"*50)
    print(f"  Trees per Acre (TPA):      {metrics['tpa']:>10.0f}")
    print(f"  Basal Area (sq ft/acre):   {metrics['basal_area']:>10.1f}")
    print(f"  Quadratic Mean DBH (in):   {metrics['qmd']:>10.1f}")
    print(f"  Mean DBH (in):             {metrics['mean_dbh']:>10.1f}")
    print(f"  Top Height (ft):           {metrics['top_height']:>10.1f}")
    print(f"  Mean Height (ft):          {metrics['mean_height']:>10.1f}")
    print(f"-"*50)
    print(f"{'Density Metrics':^50}")
    print(f"-"*50)
    print(f"  Stand Density Index:       {metrics['sdi']:>10.0f}")
    print(f"  Max SDI:                   {metrics['max_sdi']:>10.0f}")
    print(f"  Relative SDI:              {metrics['relsdi']:>10.2f}")
    print(f"  Crown Competition Factor:  {metrics['ccf']:>10.0f}")
    print(f"-"*50)
    print(f"{'Volume Metrics':^50}")
    print(f"-"*50)
    print(f"  Total Cubic Volume:        {metrics['volume']:>10.0f} cu ft/acre")
    print(f"  Merchantable Volume:       {metrics['merchantable_volume']:>10.0f} cu ft/acre")
    print(f"  Board Feet:                {metrics['board_feet']:>10.0f} BF/acre")
    print(f"  Stand Age:                 {metrics['age']:>10} years")
    print(f"{'='*50}\n")
    
    return metrics

# Display initial metrics
initial_metrics = display_metrics(stand, "Initial Stand Conditions")

## Step 8: Simulate Growth

Now let's simulate 25 years of forest growth using the FVS Southern variant growth model.

In [ ]:
# Simulate 25 years of growth
SIMULATION_YEARS = 25

print(f"Simulating {SIMULATION_YEARS} years of growth...")
stand.grow(years=SIMULATION_YEARS)
print("Growth simulation complete!")

# Display final metrics
final_metrics = display_metrics(stand, f"Final Stand Conditions (Year {SIMULATION_YEARS})")

In [ ]:
# Compare before and after
print("\n" + "="*60)
print(f"{'GROWTH SUMMARY':^60}")
print("="*60)
print(f"{'Metric':<30} {'Initial':>12} {'Final':>12} {'Change':>12}")
print("-"*60)

comparisons = [
    ("Trees per Acre", "tpa", ".0f"),
    ("Basal Area (sq ft)", "basal_area", ".1f"),
    ("QMD (inches)", "qmd", ".1f"),
    ("Top Height (ft)", "top_height", ".1f"),
    ("Total Volume (cu ft)", "volume", ".0f"),
    ("Board Feet", "board_feet", ".0f"),
]

for label, key, fmt in comparisons:
    init = initial_metrics[key]
    final = final_metrics[key]
    change = final - init
    pct = (change / init * 100) if init > 0 else 0
    print(f"{label:<30} {init:>12{fmt}} {final:>12{fmt}} {change:>+10{fmt}} ({pct:+.0f}%)")

print("="*60)
print(f"\nMean Annual Volume Increment: {(final_metrics['volume'] - initial_metrics['volume']) / SIMULATION_YEARS:.1f} cu ft/acre/year")

## Step 9: Generate Yield Table

A yield table shows projected stand development over time. This is useful for forest management planning.

In [ ]:
# Generate yield table for 50 years
yield_records = stand.generate_yield_table(
    years=50,
    period_length=5,
    stand_id="NC_LOBLOLLY",
    start_year=2025
)

# Convert to Polars DataFrame for display
yield_data = {
    "Year": [r.Year for r in yield_records],
    "Age": [r.Age for r in yield_records],
    "TPA": [r.TPA for r in yield_records],
    "BA": [r.BA for r in yield_records],
    "QMD": [r.QMD for r in yield_records],
    "TopHt": [r.TopHt for r in yield_records],
    "TCuFt": [r.TCuFt for r in yield_records],
    "MCuFt": [r.MCuFt for r in yield_records],
    "BdFt": [r.BdFt for r in yield_records],
    "MAI": [r.MAI for r in yield_records],
}
yield_df = pl.DataFrame(yield_data)

print("Yield Table (50 Years, 5-Year Periods)")
print("="*80)
print(yield_df)

## Step 10: Visualize Growth Projections

Let's create visualizations of the stand development over time.

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle(f'Loblolly Pine Stand Growth Projection\nSite Index: {stand.site_index} ft', 
             fontsize=14, fontweight='bold')

# Volume over time
ax1 = axes[0, 0]
ax1.plot(yield_df["Age"], yield_df["TCuFt"], 'b-', linewidth=2, label='Total Volume')
ax1.plot(yield_df["Age"], yield_df["MCuFt"], 'g--', linewidth=2, label='Merchantable Volume')
ax1.set_xlabel('Stand Age (years)')
ax1.set_ylabel('Volume (cu ft/acre)')
ax1.set_title('Volume Development')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Basal Area and TPA
ax2 = axes[0, 1]
ax2.plot(yield_df["Age"], yield_df["BA"], 'r-', linewidth=2)
ax2.set_xlabel('Stand Age (years)')
ax2.set_ylabel('Basal Area (sq ft/acre)', color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax2_twin = ax2.twinx()
ax2_twin.plot(yield_df["Age"], yield_df["TPA"], 'b--', linewidth=2)
ax2_twin.set_ylabel('Trees per Acre', color='b')
ax2_twin.tick_params(axis='y', labelcolor='b')
ax2.set_title('Basal Area & Stand Density')
ax2.grid(True, alpha=0.3)

# QMD and Height
ax3 = axes[1, 0]
ax3.plot(yield_df["Age"], yield_df["QMD"], 'g-', linewidth=2, label='QMD')
ax3.set_xlabel('Stand Age (years)')
ax3.set_ylabel('QMD (inches)', color='g')
ax3.tick_params(axis='y', labelcolor='g')
ax3_twin = ax3.twinx()
ax3_twin.plot(yield_df["Age"], yield_df["TopHt"], 'm--', linewidth=2)
ax3_twin.set_ylabel('Top Height (ft)', color='m')
ax3_twin.tick_params(axis='y', labelcolor='m')
ax3.set_title('Tree Size Development')
ax3.grid(True, alpha=0.3)

# Mean Annual Increment
ax4 = axes[1, 1]
ax4.plot(yield_df["Age"], yield_df["MAI"], 'k-', linewidth=2)
ax4.axhline(y=max(yield_df["MAI"]), color='r', linestyle=':', label=f'Max MAI: {max(yield_df["MAI"]):.0f}')
ax4.set_xlabel('Stand Age (years)')
ax4.set_ylabel('MAI (cu ft/acre/year)')
ax4.set_title('Mean Annual Increment')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 11: Diameter Distribution

Let's visualize how the diameter distribution changes with growth.

In [ ]:
# Get current tree diameters
diameters = [tree.dbh for tree in stand.trees]

# Create histogram
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(diameters, bins=20, edgecolor='black', alpha=0.7, color='forestgreen')
ax.axvline(x=stand.calculate_qmd(), color='red', linestyle='--', linewidth=2, 
           label=f'QMD: {stand.calculate_qmd():.1f} in')
ax.axvline(x=sum(diameters)/len(diameters), color='blue', linestyle=':', linewidth=2,
           label=f'Mean DBH: {sum(diameters)/len(diameters):.1f} in')

ax.set_xlabel('DBH (inches)', fontsize=12)
ax.set_ylabel('Number of Trees', fontsize=12)
ax.set_title(f'Diameter Distribution at Age {stand.age}', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nDiameter Statistics:")
print(f"  Minimum DBH: {min(diameters):.1f} inches")
print(f"  Maximum DBH: {max(diameters):.1f} inches")
print(f"  Mean DBH: {sum(diameters)/len(diameters):.1f} inches")
print(f"  QMD: {stand.calculate_qmd():.1f} inches")

## Bonus: Try Different Plots

You can experiment with different plots from the FIA database. Here's how to select a different plot:

In [ ]:
# Function to run simulation on any plot
def simulate_plot(plot_cn, trees_df, conds_df, years=25):
    """Run FVS simulation on a specific plot."""
    # Filter to selected plot
    plot_trees = trees_df.filter(pl.col("PLT_CN") == plot_cn)
    plot_conds = conds_df.filter(pl.col("PLT_CN") == plot_cn)
    
    print(f"Plot {plot_cn}:")
    print(f"  Trees: {len(plot_trees)}")
    print(f"  Conditions: {len(plot_conds)}")
    
    # Create stand
    try:
        stand = Stand.from_fia_data(
            tree_df=plot_trees,
            cond_df=plot_conds,
            weight_by_tpa=True,
            max_trees=500
        )
        
        initial_vol = stand.get_metrics()['volume']
        stand.grow(years=years)
        final_vol = stand.get_metrics()['volume']
        
        print(f"  Site Index: {stand.site_index}")
        print(f"  Initial Volume: {initial_vol:.0f} cu ft/acre")
        print(f"  Final Volume ({years} yrs): {final_vol:.0f} cu ft/acre")
        print(f"  Volume Growth: {final_vol - initial_vol:.0f} cu ft/acre")
        
        return stand
    except Exception as e:
        print(f"  Error: {e}")
        return None

# Try the top 3 plots
print("Simulating top 3 loblolly pine plots:\n")
for i, plot_cn in enumerate(good_plots["PLT_CN"].head(3).to_list()):
    print(f"\n--- Plot {i+1} ---")
    simulate_plot(plot_cn, trees, conds)

## Bonus 1: Yield Table from Age 0 Using FIA Site Data

You can use FIA condition data to get real site characteristics (site index, forest type) and then generate a complete yield table starting from age 0. This is useful for plantation planning and comparing site productivity.

In [ ]:
# Find loblolly pine sites with site index data
pine_sites = (
    conds
    .filter(pl.col("SISP") == 131)  # Site species = loblolly pine
    .filter(pl.col("SICOND").is_not_null())
    .filter(pl.col("SICOND") > 0)
)

print(f"Found {len(pine_sites):,} loblolly pine sites with site index data")
print(f"Site index range: {pine_sites['SICOND'].min()} - {pine_sites['SICOND'].max()} ft")
print(f"Mean site index: {pine_sites['SICOND'].mean():.1f} ft")

# Get a representative high-quality site
site_index = float(pine_sites.filter(pl.col("SICOND") >= 80)["SICOND"].mean())
print(f"\nUsing site index: {site_index:.0f} ft (mean of SI >= 80 sites)")

# Initialize a planted stand from age 0 using real FIA site data
planted_stand = Stand.initialize_planted(
    trees_per_acre=500,      # Typical plantation density
    site_index=site_index,   # From FIA data
    species="LP",            # Loblolly pine
    ecounit="232"            # Coastal Plain (use "M231" for Appalachian Mountains)
)

print(f"\nInitialized plantation:")
print(f"  Initial TPA: {len(planted_stand.trees)}")
print(f"  Site Index: {planted_stand.site_index} ft (base age 25)")
print(f"  Starting Age: {planted_stand.age}")

# Generate yield table from age 0 to 50
print("\n" + "="*85)
print(f"{'YIELD TABLE - Loblolly Pine Plantation (SI=' + str(int(site_index)) + ')':^85}")
print("="*85)
print(f"{'Age':>5} {'TPA':>8} {'BA':>10} {'QMD':>8} {'Height':>10} {'Volume':>12} {'MAI':>10}")
print(f"{'(yr)':>5} {'':>8} {'(sqft/ac)':>10} {'(in)':>8} {'(ft)':>10} {'(cuft/ac)':>12} {'(cuft/ac/yr)':>10}")
print("-"*85)

ages = list(range(0, 55, 5))
yield_data_age0 = {"Age": [], "TPA": [], "BA": [], "QMD": [], "Height": [], "Volume": [], "MAI": []}

for i, target_age in enumerate(ages):
    if i > 0:
        planted_stand.grow(years=5)
    
    m = planted_stand.get_metrics()
    mai = m["volume"] / planted_stand.age if planted_stand.age > 0 else 0
    
    yield_data_age0["Age"].append(planted_stand.age)
    yield_data_age0["TPA"].append(m["tpa"])
    yield_data_age0["BA"].append(m["basal_area"])
    yield_data_age0["QMD"].append(m["qmd"])
    yield_data_age0["Height"].append(m["top_height"])
    yield_data_age0["Volume"].append(m["volume"])
    yield_data_age0["MAI"].append(mai)
    
    print(f"{planted_stand.age:>5} {m['tpa']:>8.0f} {m['basal_area']:>10.1f} {m['qmd']:>8.1f} {m['top_height']:>10.1f} {m['volume']:>12.0f} {mai:>10.1f}")

print("="*85)

## Bonus 2: Analytical Growth Function for Optimization

We can fit the **Chapman-Richards growth function** to the yield data. This provides a smooth, differentiable function that can be used for mathematical optimization (e.g., finding optimal rotation age, maximizing NPV).

The Chapman-Richards function:
$$V(t) = A \cdot (1 - e^{-kt})^c$$

Where:
- **A** = Asymptotic maximum volume (cu ft/acre)
- **k** = Growth rate parameter (1/year)  
- **c** = Shape parameter (controls inflection point)

This function is ideal because:
1. It has **biological meaning** - asymptote represents site carrying capacity
2. It's **differentiable** - we can analytically compute MAI and PAI
3. It matches the **sigmoidal growth pattern** of forests
4. It's widely used in **forestry literature**

In [ ]:
import numpy as np
from scipy.optimize import curve_fit, minimize_scalar

# Chapman-Richards growth function
def chapman_richards(t, A, k, c):
    """
    Chapman-Richards growth function.
    
    Parameters:
        t: Age (years)
        A: Asymptotic maximum volume
        k: Growth rate parameter
        c: Shape parameter
    
    Returns:
        Volume at age t
    """
    return A * (1 - np.exp(-k * t)) ** c

# Mean Annual Increment (MAI) - analytical derivative
def mai_analytical(t, A, k, c):
    """MAI = V(t) / t"""
    if t <= 0:
        return 0
    return chapman_richards(t, A, k, c) / t

# Periodic Annual Increment (PAI) - analytical derivative
def pai_analytical(t, A, k, c):
    """PAI = dV/dt (first derivative of Chapman-Richards)"""
    if t <= 0:
        return 0
    exp_term = np.exp(-k * t)
    return A * k * c * exp_term * (1 - exp_term) ** (c - 1)

# Use yield data from Bonus 1 (yield_data_age0)
ages_array = np.array(yield_data_age0["Age"], dtype=float)
volumes_array = np.array(yield_data_age0["Volume"], dtype=float)

# Filter out age 0 for fitting (avoid division issues)
mask = ages_array > 0
ages_fit = ages_array[mask]
volumes_fit = volumes_array[mask]

# Fit Chapman-Richards to the yield data
# Initial guesses: A=max_vol*1.5, k=0.05, c=3
try:
    popt, pcov = curve_fit(
        chapman_richards, 
        ages_fit, 
        volumes_fit,
        p0=[max(volumes_fit) * 1.5, 0.05, 3.0],
        bounds=([0, 0.001, 0.5], [50000, 0.5, 10]),
        maxfev=5000
    )
    A_fit, k_fit, c_fit = popt
    
    # Calculate R² (coefficient of determination)
    predicted = chapman_richards(ages_fit, *popt)
    ss_res = np.sum((volumes_fit - predicted) ** 2)
    ss_tot = np.sum((volumes_fit - np.mean(volumes_fit)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    
    print("="*70)
    print(f"{'CHAPMAN-RICHARDS FIT RESULTS':^70}")
    print("="*70)
    print(f"\nFitted Parameters:")
    print(f"  A (asymptotic max volume) = {A_fit:,.0f} cu ft/acre")
    print(f"  k (growth rate)           = {k_fit:.6f} per year")
    print(f"  c (shape parameter)       = {c_fit:.4f}")
    print(f"\nGoodness of Fit:")
    print(f"  R² = {r_squared:.6f}")
    print(f"  RMSE = {np.sqrt(ss_res / len(volumes_fit)):.1f} cu ft/acre")
    
except Exception as e:
    print(f"Fitting failed: {e}")
    # Use reasonable defaults if fitting fails
    A_fit, k_fit, c_fit = 15000, 0.04, 3.0
    r_squared = 0

In [ ]:
# Find optimal rotation age (where MAI = PAI, or equivalently, max MAI)
def neg_mai(t):
    """Negative MAI for minimization."""
    return -mai_analytical(t, A_fit, k_fit, c_fit)

result = minimize_scalar(neg_mai, bounds=(5, 100), method='bounded')
optimal_rotation = result.x
max_mai = -result.fun
volume_at_rotation = chapman_richards(optimal_rotation, A_fit, k_fit, c_fit)

print(f"\n{'OPTIMAL ROTATION AGE':^70}")
print("-"*70)
print(f"  Biological rotation age:  {optimal_rotation:.1f} years")
print(f"  Maximum MAI:              {max_mai:.1f} cu ft/acre/year")
print(f"  Volume at rotation:       {volume_at_rotation:,.0f} cu ft/acre")
print("-"*70)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Volume curve with fitted function
ax1 = axes[0]
t_smooth = np.linspace(1, 60, 200)
v_smooth = chapman_richards(t_smooth, A_fit, k_fit, c_fit)

ax1.scatter(ages_fit, volumes_fit, s=100, c='blue', label='PyFVS Yield Table', zorder=5)
ax1.plot(t_smooth, v_smooth, 'r-', linewidth=2, label=f'Chapman-Richards Fit (R²={r_squared:.4f})')
ax1.axvline(x=optimal_rotation, color='green', linestyle='--', linewidth=2, 
            label=f'Optimal Rotation: {optimal_rotation:.0f} yrs')
ax1.axhline(y=A_fit, color='gray', linestyle=':', alpha=0.5, 
            label=f'Asymptote: {A_fit:,.0f} cu ft')

ax1.set_xlabel('Stand Age (years)', fontsize=12)
ax1.set_ylabel('Volume (cu ft/acre)', fontsize=12)
ax1.set_title('Volume Growth: FVS Data vs Analytical Function', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 60)
ax1.set_ylim(0, max(v_smooth) * 1.1)

# Right plot: MAI and PAI curves
ax2 = axes[1]
mai_smooth = [mai_analytical(t, A_fit, k_fit, c_fit) for t in t_smooth]
pai_smooth = [pai_analytical(t, A_fit, k_fit, c_fit) for t in t_smooth]

ax2.plot(t_smooth, mai_smooth, 'b-', linewidth=2, label='MAI (Mean Annual Increment)')
ax2.plot(t_smooth, pai_smooth, 'r-', linewidth=2, label='PAI (Periodic Annual Increment)')
ax2.axvline(x=optimal_rotation, color='green', linestyle='--', linewidth=2,
            label=f'MAI=PAI @ {optimal_rotation:.0f} yrs')
ax2.scatter([optimal_rotation], [max_mai], s=150, c='green', marker='*', zorder=5,
            label=f'Max MAI: {max_mai:.1f} cu ft/ac/yr')

ax2.set_xlabel('Stand Age (years)', fontsize=12)
ax2.set_ylabel('Increment (cu ft/acre/year)', fontsize=12)
ax2.set_title('MAI and PAI: Finding Optimal Rotation', fontsize=14, fontweight='bold')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 60)
ax2.set_ylim(0, max(pai_smooth) * 1.1)

plt.tight_layout()
plt.show()

### Using the Analytical Function for Economic Optimization

With the fitted Chapman-Richards function, we can optimize for economic objectives like **Net Present Value (NPV)** or **Land Expectation Value (LEV)**.

In [ ]:
# Economic parameters (customize for your analysis)
STUMPAGE_PRICE = 25.0      # $/cu ft (or adjust for your units)
PLANTING_COST = 200.0      # $/acre
ANNUAL_MGMT_COST = 10.0    # $/acre/year
DISCOUNT_RATE = 0.05       # 5% real discount rate

def npv(rotation_age, A, k, c, price, plant_cost, annual_cost, r):
    """
    Net Present Value for a single rotation.
    
    NPV = (P × V(T) × e^(-rT)) - C₀ - ∫₀ᵀ M × e^(-rt) dt
    
    Where:
        P = stumpage price
        V(T) = volume at rotation age T
        C₀ = planting cost
        M = annual management cost
        r = discount rate
    """
    T = rotation_age
    volume = chapman_richards(T, A, k, c)
    
    # Discounted harvest revenue
    harvest_pv = price * volume * np.exp(-r * T)
    
    # Discounted annual costs (integral of M*e^(-rt) from 0 to T)
    if r > 0:
        annual_costs_pv = annual_cost * (1 - np.exp(-r * T)) / r
    else:
        annual_costs_pv = annual_cost * T
    
    return harvest_pv - plant_cost - annual_costs_pv

def lev(rotation_age, A, k, c, price, plant_cost, annual_cost, r):
    """
    Land Expectation Value (Faustmann formula) - infinite rotations.
    
    LEV = NPV / (1 - e^(-rT))
    """
    T = rotation_age
    single_npv = npv(T, A, k, c, price, plant_cost, annual_cost, r)
    
    if r > 0 and T > 0:
        return single_npv / (1 - np.exp(-r * T))
    return single_npv

# Find economically optimal rotation age (maximize LEV)
def neg_lev(T):
    return -lev(T, A_fit, k_fit, c_fit, STUMPAGE_PRICE, PLANTING_COST, ANNUAL_MGMT_COST, DISCOUNT_RATE)

result_econ = minimize_scalar(neg_lev, bounds=(10, 80), method='bounded')
econ_rotation = result_econ.x
max_lev = -result_econ.fun

print("="*70)
print(f"{'ECONOMIC OPTIMIZATION RESULTS':^70}")
print("="*70)
print(f"\nParameters:")
print(f"  Stumpage price:      ${STUMPAGE_PRICE:.2f}/cu ft")
print(f"  Planting cost:       ${PLANTING_COST:.2f}/acre")
print(f"  Annual mgmt cost:    ${ANNUAL_MGMT_COST:.2f}/acre/year")
print(f"  Discount rate:       {DISCOUNT_RATE*100:.1f}%")
print(f"\nOptimal Solutions:")
print(f"  Biological rotation (max MAI):    {optimal_rotation:.1f} years")
print(f"  Economic rotation (max LEV):      {econ_rotation:.1f} years")
print(f"  Maximum LEV:                      ${max_lev:,.0f}/acre")
print(f"  Volume at economic rotation:      {chapman_richards(econ_rotation, A_fit, k_fit, c_fit):,.0f} cu ft/acre")
print("="*70)

# Compare rotation ages across discount rates
print(f"\n{'SENSITIVITY ANALYSIS: Rotation Age vs Discount Rate':^70}")
print("-"*70)
print(f"{'Discount Rate':>15} {'Economic Rotation':>20} {'LEV':>15} {'Volume':>15}")
print("-"*70)

for rate in [0.02, 0.03, 0.04, 0.05, 0.06, 0.08, 0.10]:
    def neg_lev_rate(T):
        return -lev(T, A_fit, k_fit, c_fit, STUMPAGE_PRICE, PLANTING_COST, ANNUAL_MGMT_COST, rate)
    
    res = minimize_scalar(neg_lev_rate, bounds=(10, 100), method='bounded')
    rot = res.x
    lev_val = -res.fun
    vol = chapman_richards(rot, A_fit, k_fit, c_fit)
    
    print(f"{rate*100:>14.0f}% {rot:>18.1f} yrs ${lev_val:>13,.0f} {vol:>14,.0f}")

print("-"*70)
print("Note: Higher discount rates favor shorter rotations")

In [ ]:
# Export the fitted parameters for use in other applications
print("="*70)
print(f"{'ANALYTICAL FUNCTION FOR EXTERNAL USE':^70}")
print("="*70)
print(f"""
# Python/NumPy implementation:
import numpy as np

def volume(t):
    '''Volume (cu ft/acre) at age t (years) for SI={int(site_index)} loblolly pine'''
    A = {A_fit:.4f}  # Asymptotic max volume
    k = {k_fit:.8f}  # Growth rate
    c = {c_fit:.6f}  # Shape parameter
    return A * (1 - np.exp(-k * t)) ** c

def mai(t):
    '''Mean Annual Increment at age t'''
    return volume(t) / t if t > 0 else 0

def pai(t):
    '''Periodic Annual Increment (instantaneous growth rate) at age t'''
    A, k, c = {A_fit:.4f}, {k_fit:.8f}, {c_fit:.6f}
    exp_term = np.exp(-k * t)
    return A * k * c * exp_term * (1 - exp_term) ** (c - 1)

# Example usage:
# volume(25)  # Volume at age 25
# mai(25)     # Mean annual increment at age 25
# pai(25)     # Current annual growth at age 25
""")

print("-"*70)
print("Mathematical notation:")
print(f"  V(t) = {A_fit:.2f} × (1 - e^(-{k_fit:.6f}×t))^{c_fit:.4f}")
print(f"  MAI(t) = V(t) / t")
print(f"  PAI(t) = dV/dt = {A_fit:.2f} × {k_fit:.6f} × {c_fit:.4f} × e^(-{k_fit:.6f}×t) × (1 - e^(-{k_fit:.6f}×t))^{c_fit-1:.4f}")
print("="*70)

## Summary

This notebook demonstrated the complete workflow for forest growth simulation:

1. **Downloaded** FIA database using PyFIA
2. **Loaded** tree and condition data for North Carolina
3. **Selected** a loblolly pine-dominated plot
4. **Created** a PyFVS Stand from FIA data
5. **Simulated** 25 years of growth
6. **Generated** yield tables and visualizations

### Key Takeaways

- **PyFIA** provides easy access to FIA databases with filtering and querying capabilities
- **PyFVS** simulates forest growth using the FVS Southern variant equations
- The `Stand.from_fia_data()` method handles all the conversion between FIA and FVS formats
- Growth simulations can be used for forest management planning and yield projections

### Next Steps

- Try different states (change `!pyfia download <state>` and `STATE_FIPS`)
- Experiment with different species (change `TARGET_SPECIES`)
- Apply thinning treatments using `stand.thin_from_below()` or `stand.thin_from_above()`
- Compare managed vs. unmanaged stand development

### Resources

- [PyFIA Documentation](https://github.com/mihiarc/pyfia)
- [PyFVS Documentation](https://github.com/mihiarc/pyfvs)
- [FIA DataMart](https://apps.fs.usda.gov/fia/datamart/)
- [FVS Documentation](https://www.fs.usda.gov/fvs/)